<center><h1>DSCI 552 HW3</h1></center>
<br>
<center><font size="4">Time Series Classification</font></center>
<br />
<table align="center">
    <tbody>
    <tr><td>Name</td><td>Parth Patel</td></tr>
    <tr><td>USC ID</td><td>5228107917</td></tr>
     <tr><td>Github Username</td><td>parthpar-usc</td></tr>
    <tr><td>Email</td><td>parthpar@usc.edu</td></tr>
    </tbody>
</table>

## 1.Time Series Classification Part 1: Feature Creation/Extraction

#### (a)Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29. The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity. Each file containis 6 time series collected from activities of the same person, which are called avg_rss12, var_rss12, avg_rss13, var_rss13,vg_rss23, and ar_rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

#### Import required packages

In [1]:
import os
import shutil
import numpy as np
from scipy.stats import bootstrap
import numpy as np
import pandas as pd
from sklearn.utils import resample



#### (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,2, and 3 in other folders as test data and other datasets as train data.

In [2]:
base_path = '../data/AReM'

train_data = []
test_data = []

def load_data(file_path, activity, test=False):
    try:
        df = pd.read_csv(file_path, skiprows=4)
    except pd.errors.ParserError:
        try:
            df = pd.read_csv(file_path, skiprows=4)
        except Exception as e:
            print(f"Failed to read {file_path} on second attempt: {e}")
            return
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return
    
    if test:
        test_data.append({'activity': activity, 'filename': os.path.basename(file_path), 'data': df})
    else:
        train_data.append({'activity': activity, 'filename': os.path.basename(file_path), 'data': df})

for activity in os.listdir(base_path):
    activity_path = os.path.join(base_path, activity)
    if os.path.isdir(activity_path):
        filenames = sorted(os.listdir(activity_path))
        
        if activity in ['bending1', 'bending2']:
            for filename in filenames[:2]:
                file_path = os.path.join(activity_path, filename)
                load_data(file_path, activity, test=True)
            for filename in filenames[2:]:
                file_path = os.path.join(activity_path, filename)
                load_data(file_path, activity)
                
        else:
            for filename in filenames[:3]:
                file_path = os.path.join(activity_path, filename)
                load_data(file_path, activity, test=True)
            for filename in filenames[3:]:
                file_path = os.path.join(activity_path, filename)
                load_data(file_path, activity)


def convert_and_flatten_data(data_list):
    flattened_data = []
    for item in data_list:
        df = item['data']
        for column in df.columns:
            try:
                df[column] = pd.to_numeric(df[column], errors='coerce')
            except Exception as e:
                print(f"Error converting column {column} to numeric: {e}")
                continue 
        
        numeric_df = df.select_dtypes(include=['number']) 
        mean_values = numeric_df.mean().to_dict()
        
        mean_values['activity'] = item['activity']
        mean_values['filename'] = item['filename']
        flattened_data.append(mean_values)
    
    return pd.DataFrame(flattened_data)

train_df = convert_and_flatten_data(train_data)
test_df = convert_and_flatten_data(test_data)
                                                                                                                                        
len(test_data)


19

### (c) Feature Extraction
### Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features

#### i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

The features that are commonly used for time series classification are as follows:

Min: It is the minimum value in a dataset.

Max: It is the maximum value in a dataset.

Mean: It is the average of the values in a dataset.

Median: It is the middle value which divides the upper half and lower half of the dataset.

Standard Deviation: It is a measure of how dispersed the data is in relation to the mean.

1st Quartile: It the value under which 25% if the data points are found in a dataset.

3rd Quartile: It the value under which 75% if the data points are found in a dataset.

Interquartile Range: It is the range between 25% and 75%, where the middle 50% of the data points lie.

Range: It is the difference between the min and max values of a dataset.

Variance: It is mean squared difference between each data point and the centre of the distribution measured by the mean.

#### ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [3]:
def extract_features(series):
    return [
        series.min(),
        series.max(),
        series.mean(),
        series.median(),
        series.std(),
        series.quantile(0.25),
        series.quantile(0.75)
    ]

def load_data(file_path):
    try:
        df = pd.read_csv(file_path, skiprows=5, header=None)
        df = df.drop(df.columns[0], axis=1)
        return df
    except pd.errors.ParserError:
        print(f"Parser error when reading {file_path}")
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
    return None

feature_vectors = []

base_path = '../data/AReM'

instance_counter = 1

for activity in os.listdir(base_path):
    activity_path = os.path.join(base_path, activity)
    
    if os.path.isdir(activity_path):
        filenames = sorted(os.listdir(activity_path)) 
        
        for filename in filenames:
            file_path = os.path.join(activity_path, filename)
            df = load_data(file_path)
            if df is not None:
                features = [instance_counter]
                for column in df.columns:
                    features.extend(extract_features(df[column]))
                
                feature_vectors.append(features)
                instance_counter += 1

features_df = pd.DataFrame(feature_vectors, columns=[
    'Instance', 'min1', 'max1', 'mean1', 'median1', 'std1', '1st_quart1', '3rd_quart1',
    'min2', 'max2', 'mean2', 'median2', 'std2', '1st_quart2', '3rd_quart2',
    'min3', 'max3', 'mean3', 'median3', 'std3', '1st_quart3', '3rd_quart3',
    'min4', 'max4', 'mean4', 'median4', 'std4', '1st_quart4', '3rd_quart4',
    'min5', 'max5', 'mean5', 'median5', 'std5', '1st_quart5', '3rd_quart5',
    'min6', 'max6', 'mean6', 'median6', 'std6', '1st_quart6', '3rd_quart6'
])

features_df


,Instance,min1,max1,mean1,median1,std1,1st_quart1,3rd_quart1,min2,max2,...,std5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std6,1st_quart6,3rd_quart6
0,1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
1,2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.3000
2,3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.9400
3,4,33.00,47.75,42.179812,43.50,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,26.50,44.33,36.687292,36.00,3.529404,34.25,39.3725,0.0,12.89,...,2.978238,14.6700,18.50,0.0,8.19,3.073313,2.690,1.629675,1.9125,4.0875
84,85,25.33,45.00,37.114313,36.25,3.710385,34.50,40.2500,0.0,10.84,...,2.847876,14.7500,18.50,0.0,9.50,3.076354,2.770,1.824534,1.7000,4.0375
85,86,26.75,44.75,36.863375,36.33,3.555787,34.50,39.7500,0.0,11.68,...,2.655906,15.0000,18.67,0.0,8.81,2.773313,2.590,1.569919,1.6400,3.6325
86,87,26.25,44.25,36.957458,36.29,3.434863,34.50,40.2500,0.0,8.64,...,2.851673,14.0000,18.25,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


### iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature

In [4]:

table_headers = ["Feature", "Std Dev", "Low", "High"]
table_data = []

def bootstrap_std_confidence_interval(feature_data, n_resamples=1000, conf_level=0.9):
    bootstrapped_stds = []
    for _ in range(n_resamples):
        sample = resample(feature_data)
        bootstrapped_stds.append(np.std(sample, ddof=1)) 
    alpha = 1 - conf_level
    lower_percentile = alpha / 2.0 * 100
    upper_percentile = (1 - alpha / 2.0) * 100
    confidence_low = np.percentile(bootstrapped_stds, lower_percentile)
    confidence_high = np.percentile(bootstrapped_stds, upper_percentile)
    return np.std(feature_data, ddof=1), confidence_low, confidence_high  

for col in features_df.columns:
    feature_data = features_df[col]
    std_dev, low, high = bootstrap_std_confidence_interval(feature_data)  
    table_data.append([col, std_dev, low, high])  

results_df = pd.DataFrame(table_data, columns=table_headers)

results_df


,Feature,Std Dev,Low,High
0,Instance,25.547342,23.333177,27.361909
1,min1,9.624011,8.334032,10.817845
2,max1,4.207745,3.085972,5.044429
3,mean1,5.276431,4.606115,5.799321
4,median1,5.386624,4.765097,5.984648
5,std1,1.771282,1.566854,1.948459
6,1st_quart1,6.127846,5.582073,6.644598
7,3rd_quart1,5.031028,4.208553,5.766266
8,min2,0.000000,0.000000,0.000000
9,max2,5.059656,4.592262,5.397721


### iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

### Mean (meanX): Highlights the average behavior of the time series, revealing the central trend across an observation period.

### Standard Deviation (stdX): Indicates the level of fluctuation around the mean, with higher values suggesting more erratic behavior and lower values pointing to stability.

### Maximum (maxX) / Minimum (minX): Identifies extreme peaks and troughs, crucial for spotting exceptional events and understanding the range of variation.

### 2. I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. $ Y = β0 +β1X +β2X^2 +β3X^3 +ε $.
### (a) Suppose that the true relationship between X and Y is linear, i.e. $Y = β0 + β1X + ε$. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

### If the true relationship is linear, the training RSS for the cubic regression will likely be lower or the same as for the linear regression due to its flexibility to fit the data more closely, potentially including noise.

### (b) Answer (a) using test rather than training RSS.

### For the test RSS, the linear regression should perform better if the true relationship is linear, as the cubic regression may overfit the training data and thus perform worse on unseen test data.

### (c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

### If the true relationship between $X$ and $Y$ is not linear, then the linear regression model will not be the correct specification, and the cubic regression may capture the relationship better if the true form is closer to cubic. In this case, we would expect the training RSS for the cubic regression to be lower than for the linear regression, as the cubic model can adjust to more complex relationships in the data.

### (d) Answer (c) using test rather than training RSS.

### For the test RSS, if the true relationship is non-linear, the cubic regression is expected to have a lower test RSS, indicating better generalization and fit to the true underlying pattern.